In [5]:
import cadquery as cq

# Create a sloped box: 50x50x10, then rotate 20° about X to tilt the top
sloped_box = (
    cq.Workplane("XY")
    .box(50, 50, 10)
    .rotate((0, 0, 0), (1, 0, 0), 20)  # rotate around X axis
)

# Export
cq.exporters.export(sloped_box, 'sloped_box.step')

In [1]:
import cadquery as cq

# Create a box at the origin (centered by default)
box = cq.Workplane("XY").box(50, 50, 10)  # width, depth, height

# Export to a STEP file
cq.exporters.export(box, 'box_50x50.step')


In [2]:
model = cq.importers.importStep('box_50x50.step')

In [ ]:
import cadquery as cq

# Load your sloped STEP file
model = cq.importers.importStep("sloped_box.step")

# Get bounding box
bbox = model.val().BoundingBox()
xmin, xmax = bbox.xmin, bbox.xmax
ymin, ymax = bbox.ymin, bbox.ymax
zmin = bbox.zmin  # This is the minimum Z value (bottom of the bounding box)

# Parameters
spacing = 5.0
divot_radius = 1.0
divot_depth = 1.0

# Build divot geometry on the actual 3D surface
divots = cq.Workplane("XY")

for x in range(int(xmin), int(xmax) + 2, int(spacing)):
    for y in range(int(ymin), int(ymax)+ 2 , int(spacing)):
        # Create a ray (small box pointing downward) that starts below the object's surface
        ray = cq.Solid.makeBox(0.1, 0.1, 100).translate((x, y, zmin - 50))

        # Perform intersection to find where the ray meets the surface
        intersection = model.intersect(ray)
        
        try:
            # Get the highest Z value in the intersected shape
            z_top = intersection.val().BoundingBox().zmax

            # Create a cylinder that extends above the surface
            cylinder = cq.Workplane("XY").cylinder(divot_depth + 2.0, divot_radius)  # Extend the cylinder above the surface

            # Create a sphere at the bottom of the cylinder (to round out the divot)
            # Position the sphere so that its center is at the bottom of the cylinder
            sphere = cq.Workplane("XY").sphere(divot_radius).translate((0, 0, -2 * divot_depth))

            # Combine the cylinder and sphere into one shape (cylinder with a spherical bottom)
            divot = cylinder.union(sphere)
            # Translate the divot to the correct position, starting from the object's surface
            divot = divot.translate((x, y, z_top))

            # Add to combined divots
            divots = divots.union(divot)
        except Exception as e:
            print('skipping divot at ({}, {}): {}'.format(x, y, e))
            continue

# Subtract divots from original model
modified = model.cut(divots)

# Export result
cq.exporters.export(modified, 'divoted_sloped_box.step')


AttributeError: 'Workplane' object has no attribute 'isEmpty'